### Dual variables of the Problem

When we work with exact models in Pyomo we can have access to the dual variables of the problem (Lagrange multipliers or dual variables)

*Quickly introduction to dual variables*

Suppose you have a problem where we wish to minimize the total cost of production of two machines. And we have the following parameters and variables in the problem.

- $C_1$: cost of production of machine 1 (parameter)
- $C_2$: cost of production of machine 2 (parameter)
- $N_1$: number of products from machine 1 (variable)
- $N_2$: number of products from machine 2 (variable)
- $N_{max1}$: capacity of production of machine 1 (parameter)
- $N_{max2}$: capacity of production of machine 2 (parameter)
- $D$: total demand of products (parameter)


So, our problem is modelled as:

$$
\begin{alignat*}{3}
\text{minimize  }  & C_1*N_1 + C_2*N_2 && \\
\text{subject to  }
& N_1 + N_2 = D\\
& N_1 <= N_{max1}\\
& N_2 <= N_{max2}
\end{alignat*}
$$



In a mathematical model, each constraint has a dual variable:
- $N_1 + N_2 = D$ [lambda]
- $N_1 <= N_{max1}$ [phi1]
- $N_2 <= N_{max2}$ [phi2]



In the example above.

If **phi1** is 0, it means that N1 < Nmax1. And this constraint is not in its boundary. 
If **phi1** is different than 0, it means that N1 = Nmax1. In other words, the constraint is in its boundary. And **phi1** is the "shadow price".

What does this "shadow price" mean? It means that the value of phi1 represents how much you can improve in the objective function if you "loosen/relax" the constraint a little. This will be very clear for you in the example below.

The same is applied to **phi2**; however, the interpretation of lambda (dual variable for = constraint) is quite more complex and not so important as **phi1** and **phi2**.


In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
m = pe.ConcreteModel()

#variables
m.N1 = pe.Var()
m.N2 = pe.Var()
N1 = m.N1
N2 = m.N2

#parameters
C1 = 100
C2 = 300
Nmax1 = 31
Nmax2 = 100
D = 50


#objective function and constraints
m.obj = pe.Objective(expr = C1*N1+C2*N2)
m.C1 = pe.Constraint(expr = N1+N2 == D)
m.C2 = pe.Constraint(expr = N1 <= Nmax1)
m.C3 = pe.Constraint(expr = N2 <= Nmax2)


#solving model
m.dual = pe.Suffix(direction=pe.Suffix.IMPORT)
opt = po.SolverFactory('glpk')
m.results = opt.solve(m)


In [4]:
#dual variables
print('-'*20 + ' MODEL ' + '-'*20)
m.pprint() 
 
#prints the model
print('\n\n'+'-'*20 + ' DUAL VARIABLES ' + '-'*20)
m.dual.pprint() 
 
#prints the dual variables
print('\n\n'+'-'*20 + ' Value of ObjFun ' + '-'*20)
print('Objective Function is %.2f' % (pe.value(m.obj)))
 

-------------------- MODEL --------------------
2 Var Declarations
    N1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  31.0 :  None : False : False :  Reals
    N2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  19.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 100*N1 + 300*N2

3 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  50.0 : N1 + N2 :  50.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :   N1 :  31.0 :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :   N2 : 100.0 :   True

1 Suffix Declarations
    dual : Dire

In [5]:
#access a single dual variable
mydual = m.dual[m.C2]

Now the dual variables can be accessed typing m.dual.pprint().

This is very important.

The dual variable for C2 (-200) means that if you increase the limit of constraint 2 by 1 unit (so 31), the objective function will be 8,800 (-200 from the previous one).




Since the dual variable for C3 is 0, this is a constraint that is not in its boundary. In other words, the solution found is NOT is the boundary of this constraint. If you wish to know how far it is from the bounder, use: m.C3.uslack() or m.C3.lslack() to see the upper and lower slacks, which mean, how far the constraint is from its boundary.

